In [ ]:
import cuentitas
from cuentitas import a0VV, a1VVF1, a1VVF2
import auxiliaresGNSS as au

from numpy import pi, sqrt, sin, cos, exp, log10, array, real, conj
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import interpolate
import seaborn as sns
import pandas as pd
import glob
import theano.tensor as TT
import time

import pymc3 as pm
import random 
sns.set()

In [ ]:
import theano.tensor as T

In [ ]:
# s0 = patron(epsilon1,epsilon2,de,ssup,sinf,altura)
def patron(ep,s,ZA):


    landa = 0.19
    k0 = 2*np.pi/landa
    phi = 0
    ph = phi
    th = thi

    # sigmaFran(1, ep, 0.1, 0.001, 0.01, s, l, incAng[j], landa[k])
    # s0 = au.generadorS0_Fran(ep1,ep2,d,s1,s2,ZA) #2 capas
    s0 = au.generadorS0(thi,1,ep,0.1,0.01,s,ZA) #1 capa

    nn = len(thi)
    
    a = TT.zeros(nn)

    for i in range(nn):
        a = TT.set_subtensor(a[i], s0[i])
        
    return a

In [ ]:
def patron_ruidoso(ep,s,ZA):

    RANDOM_SEED = 58

    landa = 0.19
    k0 = 2*np.pi/landa
    phi = 0
    ph = phi
    th = thi

    # sigmaFran(1, ep, 0.1, 0.001, 0.01, s, l, incAng[j], landa[k])
    # s0 = au.generadorS0_Fran(ep1,ep2,d,s1,s2,ZA) #2 capas
    s0 = au.generadorS0ruido(thi,1,ep,0.1,0.01,s,ZA,noise) #1 capa

    nn = len(thi)
    
    a = TT.zeros(nn)

    for i in range(nn):
        a = TT.set_subtensor(a[i], s0[i])
        
    return a

In [ ]:
# def modelo(medidos,errores,samples,thi): 
def modelo(medidos,errores,samples): 

    RANDOM_SEED = 58
    ranges = {}

    ''' DOS CAPAS
    ranges['ep1'] = 4, 40
    ranges['ep2'] = 4, 40
    ranges['d'] = 0.03, 0.15
    ranges['s1'] = 0.005, 0.02
    ranges['s2'] = 0.005, 0.02
    ranges['Za'] = 3, 3.5
    '''

    ranges['ep'] = 4, 40
    ranges['s'] = 0.001, 0.02
    ranges['Za'] = 2.99, 3.01 #poca incerteza

    with pm.Model() as model:
        ep = pm.Uniform('ep', ranges['ep'][0], ranges['ep'][1])
        s = pm.Uniform('s', ranges['s'][0], ranges['s'][1])
        Za = pm.Uniform('Za', ranges['Za'][0], ranges['Za'][1])

    def f(ep = ep,
          s = s,
          Za = Za):
        # return patron(ep1,ep2,d,s1,s2,Za) #2 capas
        return patron(ep,s,Za)
    
    with model:
        function_pm = pm.Deterministic('s0f', f())
        observations = pm.Normal( "obs",  mu=real(function_pm), sigma=errores, observed=medidos)
        trace = pm.sample_smc(samples, parallel=True)

    with model:
        ppc = pm.sample_posterior_predictive(trace, var_names=["ep", "s", "obs"], random_seed=RANDOM_SEED, samples=10)
        
    return trace, ppc, model

In [ ]:
# Trabajo en angulos medidos desde el azimut, los angulos de elevacion (gps) estan medidos desde el suelo. VOY A LLAMAR ELEVACION A LOS MEDIDOS DESDE AZIMUT
# Las cotas que suponemos para nuestro suelo es entre 8º y 28º de elevacion y corresponden a ep entre 4 y 40. 

# inputs
epsilon = 15.5
ssup = 0.011
altura = 3.0

entrada = [epsilon,ssup,altura]

# voy a analizar como varia al meter mas o menos mediciones, es decir, cambiando la discretizacion en los angulos
discretizacion = 100
thi = np.linspace(62*pi/180,82*pi/180, discretizacion)

# s0 = au.generadorS0(elevacion,epsilon1,epsilon2,de,ssup,sinf,altura)
s0 = patron(epsilon,ssup,altura)
plt.plot(thi*180/pi,s0.eval())

In [ ]:
# #agrego ruido
# noise = 10 #Valor en porcentaje del ruido que quiero ej: 10 = 10% de ruido en la medicion
# noise = 15 #Valor en porcentaje del ruido que quiero ej: 10 = 10% de ruido en la medicion
noise = 5 #Valor en porcentaje del ruido que quiero ej: 10 = 10% de ruido en la medicion
# noise = 1 #Valor en porcentaje del ruido que quiero ej: 10 = 10% de ruido en la medicion


pert = patron_ruidoso(epsilon,ssup,altura)
plt.plot(thi*180/pi,pert.eval() ,c='r')
plt.plot(thi*180/pi,s0.eval(),linewidth=1,c='b')

In [ ]:
# variance is the average of the squared differences from the mean.
var = np.mean((pert.eval() - s0.eval())**2)
sd = var**0.5
print('sd = ' + str(sd))

In [ ]:
plt.hist(pert.eval() - s0.eval())

In [ ]:
type(pert.eval())

In [ ]:
t0 = time.time()

# medidos = s0
medidos = pert.eval()


# sig = 0.15 #default

# Parameters

#     mu: float

#         Mean.
#     sigma: float

#         Standard deviation (sigma > 0) (only required if tau is not specified).
#     tau: float

#         Precision (tau > 0) (only required if sigma is not specified).


# sig = 5*sd # ruido que anda
sig = sd

# errores = np.asarray(len(s0.eval())*[sig]) #ojo saque esto
errores = sig


muestreo = 3000

# traza = modelo(medidos,errores,muestreo,elevacion)
traza, ppc, model = modelo(medidos,errores,muestreo)

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(16, 16), dpi=150)

plotear = np.stack((traza['ep'],traza['s'],traza['Za'])).T
dfplotear = pd.DataFrame(plotear, columns = ['ep','s','Za'])

ep_sample_mean = traza['ep'].mean()
s_sample_mean = traza['s'].mean()
Za_sample_mean = traza['Za'].mean()

sample_mean = [ep_sample_mean, s_sample_mean, Za_sample_mean]


g = sns.PairGrid(dfplotear)
g.fig.suptitle('GNSS2022-#med-1capa:'+str(discretizacion)+'ruido='+str(noise)+'%-set4', y=1.01, fontsize = 'large')
g.map_upper(sns.histplot)
g.map_lower(sns.kdeplot, fill=True)
g.map_diag(sns.histplot, kde=True, color='.5')


for i in range(len(entrada)):
    for j in range(len(entrada)):
        if i == j:
            #ploteo el valor verdadero en las marginales
            g.axes[j,i].axvline(entrada[i], color='r', linestyle='dashed', linewidth=2) #Este me descuajeringa todo, despues veo como arreglarlo
            g.axes[j,i].axvline(sample_mean[i], color='g', linestyle='dashed', linewidth=2)
            pass
        else:
            g.axes[j,i].scatter(entrada[i], entrada[j], marker="+", c='r', s=300) #color='k', linestyle='dashed', linewidth=1)
            g.axes[j,i].scatter(sample_mean[i], sample_mean[j], marker="+", c='g', s=300) #color='k', linestyle='dashed', linewidth=1)

In [ ]:
Nombre = 'Analisis_de_error/#d='+'F-1capa'+'m='+str(muestreo)+'sigma='+'{:.2f}'.format(sd)+'-Ns='+str(noise)+'%-set4'


In [ ]:
Nombre

In [ ]:
g.savefig(Nombre+'.png', dpi=300)

t1 = time.time()
print("Tiempo de ejecución:", t1-t0, 'seg','(', (t1-t0)/3600,  'hs )')

In [ ]:
e1 = traza['ep']
# e2 = traza['ep2']
# d = traza['d']
s1 = traza['s']
# s2 = traza['s2']
Za = traza['Za']

guardar = [e1,s1,Za]

f2 = np.savetxt(Nombre+'.txt',guardar, fmt='%s', delimiter=';')

In [ ]:
# ep1 = 25
# ep2 = 33
# d = 0.1
# s1 = 0.01
# s2 = 0.015
# z = 3.15

In [ ]:
a = sns.kdeplot(traza['ep'],traza['s'], fill=True)#, cmap='cubehelix',thresh = 0,levels=100)
a.set(xlabel='ep', ylabel='s')
a.scatter(epsilon, ssup, marker="+", c='r', s=200)
fig = a.get_figure() 
# fig.savefig('Analisis_de_error/ep1ep2.png',dpi = 300)

In [ ]:
#PPC
import arviz as az

In [ ]:
az.plot_ppc(az.from_pymc3(posterior_predictive=ppc, model=model))

In [ ]:
ppc['obs'].mean(axis=0).shape

In [ ]:
from matplotlib.pyplot import figure

g1 = figure(figsize=(18, 8), dpi=80)

plt.plot(thi*180/pi,pert.eval(),linewidth=2, c='r', label='obs')

alphaPPC = 0.3

ppc_mean = ppc['obs'].mean(axis = 0)

plt.plot(thi*180/pi, ppc_mean, alpha = 0.8, linewidth=2, c='b', label='sample mean')

for i in range(10):
    plt.plot(thi*180/pi, ppc['obs'][i,:], ':k', alpha = alphaPPC,  label='sample')
    # plt.plot(thi*180/pi, ppc['obs'][0,:], ':k', alpha = alphaPPC,  label='sample #1')
    # plt.plot(thi*180/pi, ppc['obs'][4,:], ':k', alpha = alphaPPC, label='sample #2')
    # plt.plot(thi*180/pi, ppc['obs'][8,:], ':k', alpha = alphaPPC, label='sample #3')

plt.legend()

g1.savefig(Nombre+'+_ppc.png', dpi=300)

